# Entraînement et AutoML

## Dans ce Notebook, vous allez apprendre
- Qu'est-ce que l'**entraînement** ?
- Introduction aux entraîneurs, quelques-unes de leurs différences, et comment choisir lequel utiliser.
- Comment les hyperparamètres impactent les performances de l'entraînement.
- Comment utiliser AutoML pour simplifier votre processus d'entraînement.

## Qu'est-ce que l'**entraînement** ?
Avant de plonger dans le code, parlons d'abord de ce que signifie "entraîner un modèle".

Dans ML.Net, "entraîner un modèle" signifie généralement appeler `model.Fit(X)` dans ML.Net, où `X` est un `IDataView` qui inclut à la fois les caractéristiques et les étiquettes. Alors, que se passe-t-il lorsque vous appelez `Fit` ? En général, `Fit` met à jour les paramètres dans l'entraîneur afin qu'il puisse prédire une étiquette qui soit **proche** de l'étiquette réelle dans `X`, ou en d'autres termes, réduire la distance entre l'étiquette prédite et l'étiquette réelle.

En apprentissage automatique, la différence ou la distance entre l'étiquette prédite et l'étiquette réelle est généralement appelée **perte** et vous utilisez différentes mesures de perte en fonction de la tâche. Pour la classification, softmax est une mesure de perte courante. Pour la régression, l'erreur quadratique moyenne (RMSE) est une mesure de perte courante. En général, elles sont toutes des métriques pour quantifier la distance entre l'étiquette prédite et l'étiquette réelle. Dans la plupart des cas, une **perte plus faible signifie un meilleur modèle**. Pour plus d'informations, consultez le [guide des métriques d'évaluation ML.NET](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics).

Ainsi, ce que fait `Fit` est d'appliquer un algorithme à vos données pour identifier des motifs et ajuster les paramètres dans cet algorithme pour réduire la perte. Lorsque vous entraînez un modèle, vous voulez réduire sa perte pour rendre la prédiction de ce modèle plus proche de l'étiquette réelle.


In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"
#r "nuget: Microsoft.ML.AutoML"
#r "nuget: Microsoft.Data.Analysis"

// Import usings.
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;


## Les entraîneurs dans ML.Net
ML.NET fournit une variété d'entraîneurs. Vous pouvez en trouver la plupart sous le [StandardTrainersCatalog](https://docs.microsoft.com/dotnet/api/microsoft.ml.standardtrainerscatalog?view=ml-dotnet). Exemples d'entraîneurs : des entraîneurs linéaires comme `SDCA`, `Lbfgs`, `LinearSvm` et des entraîneurs non linéaires basés sur les arbres comme `FastTree`, `RandomForest` et `LightGbm`. En général, les capacités de chaque entraîneur sont différentes. Les modèles non linéaires ont parfois de meilleures performances d'entraînement (perte plus faible) que les modèles linéaires, mais cela ne signifie pas toujours qu'ils sont toujours le meilleur choix. Choisir le bon entraîneur pour construire le meilleur modèle pour vos données nécessite de nombreux essais et erreurs.

### Surapprentissage et sous-apprentissage
Le surapprentissage et le sous-apprentissage sont les deux problèmes les plus courants que vous rencontrez lors de l'entraînement d'un modèle. Le sous-apprentissage signifie que l'entraîneur sélectionné n'est pas assez performant pour ajuster le jeu de données d'entraînement et résulte généralement en une perte élevée pendant l'entraînement et un score/métrique faible sur le jeu de test. Pour résoudre ce problème, vous devez soit sélectionner un modèle plus puissant, soit effectuer davantage d'ingénierie des fonctionnalités. Le surapprentissage est l'inverse, ce qui se produit lorsque le modèle apprend trop bien les données d'entraînement. Cela résulte généralement en une perte faible pendant l'entraînement mais une perte élevée sur le jeu de test.

Une bonne analogie pour ces concepts est l'étude pour un examen. Disons que vous connaissiez à l'avance les questions et les réponses. Après avoir étudié, vous passez l'examen et obtenez un score parfait. Bonne nouvelle ! Cependant, lorsqu'on vous donne à nouveau l'examen avec les questions réarrangées et légèrement reformulées, vous obtenez un score plus bas. Cela suggère que vous avez mémorisé les réponses sans vraiment apprendre les concepts sur lesquels vous étiez évalué. C'est un exemple de surapprentissage. Le sous-apprentissage est l'inverse, où les matériaux d'étude que vous avez reçus ne représentent pas précisément ce sur quoi vous êtes évalué pour l'examen. En conséquence, vous devez deviner les réponses car vous n'avez pas suffisamment de connaissances pour répondre correctement.

### Différence entre paramètre et hyper-paramètre
En résumé, les paramètres sont internes à un entraîneur et sont mis à jour en fonction du jeu de données d'entraînement pendant le processus d'entraînement (`Fit`). Les hyper-paramètres sont externes à un entraîneur et contrôlent le processus d'entraînement. Par exemple, dans `LightGbm`, le `LearningRate` est un hyper-paramètre que vous pouvez désigner lors de la création et il contrôle les étapes de mise à jour pour le poids des nœuds de l'arbre pendant l'entraînement. Et le poids des nœuds de l'arbre est un paramètre qui est ajusté pendant le processus `Fit`.

### Optimisation des hyper-paramètres
Choisir le bon entraîneur impacte vos performances finales d'entraînement. Choisir les bons hyper-paramètres a également un impact énorme sur les performances finales de l'entraînement, en particulier pour les entraîneurs basés sur les arbres. Les hyper-paramètres sont importants car ils contrôlent comment les paramètres sont mis à jour. Par exemple, un plus grand `numberOfLeaves` dans `LightGbm` produit un modèle plus grand et permet généralement de s'adapter à un jeu de données plus complexe, mais cela peut avoir un effet contraire sur un petit jeu de données et provoquer un **surapprentissage**. À l'inverse, si le jeu de données est complexe mais que vous définissez un petit `numberOfLeaves`, cela peut nuire à la capacité de `LightGbm` à s'adapter à ce jeu de données et provoquer un **sous-apprentissage**.

Le processus de recherche de la meilleure configuration pour votre entraîneur est connu sous le nom d'optimisation des hyper-paramètres (HPO). Comme le processus de choix de votre entraîneur, il implique beaucoup d'essais et d'erreurs. Les capacités intégrées de ML automatisé (AutoML) dans ML.NET simplifient le processus de HPO.


In [ ]:
var rand = new Random(0);
var context =new MLContext(seed: 1);
var x = Enumerable.Range(-10000, 10000).Select(_x => _x * 0.1f).ToArray();
var y = x.Select(_x => 100 * _x + (rand.NextSingle() - 0.5f) * 10).ToArray();
var df = new DataFrame();
df["X"] = DataFrameColumn.Create("X", x);
df["y"] = DataFrameColumn.Create("y", y);
var trainTestSplit = context.Data.TrainTestSplit(df);
df.Head(10)


## Exemple 1 : Régression linéaire
Dans la section ci-dessous, nous allons montrer la différence entre les entraîneurs via une tâche de régression linéaire. Tout d'abord, nous ajustons le jeu de données linéaire avec l'entraîneur linéaire `SDCA`. Ensuite, nous ajustons le jeu de données linéaire avec `LightGbm`, un entraîneur non linéaire basé sur les arbres. Leurs performances sont évaluées par rapport à un jeu de test. Le code ci-dessous :
- Crée un jeu de données linéaire et le divise en ensembles d'entraînement et de test
- Crée des pipelines d'entraînement utilisant `SDCA` et `LightGbm`
- Entraîne `SDCA` et `LightGbm` sur le jeu d'entraînement linéaire, et les évalue sur le jeu de test.


In [ ]:
var sdcaPipeline = context.Transforms.Concatenate("Features", "X")
                            .Append(context.Regression.Trainers.Sdca("y"));

var lgbmPipeline = context.Transforms.Concatenate("Features", "X")
                            .Append(context.Regression.Trainers.LightGbm("y"));

var sdcaModel = sdcaPipeline.Fit(trainTestSplit.TrainSet);
var lgbmModel = lgbmPipeline.Fit(trainTestSplit.TrainSet);

// évaluation sur le jeu d'entraînement
var sdcaTrainEval = sdcaModel.Transform(trainTestSplit.TrainSet);
var sdcaTrainMetric = context.Regression.Evaluate(sdcaTrainEval, "y");

var lgbmTrainEval = lgbmModel.Transform(trainTestSplit.TrainSet);
var lgbmTrainMetric = context.Regression.Evaluate(lgbmTrainEval, "y");

Console.WriteLine($"sdca rmse sur trainset: {sdcaTrainMetric.RootMeanSquaredError}, lgbm rmse sur trainset: {lgbmTrainMetric.RootMeanSquaredError}");

// évaluation sur le jeu de test
var sdcaTestEval = sdcaModel.Transform(trainTestSplit.TestSet);
var sdcaTestMetric = context.Regression.Evaluate(sdcaTestEval, "y");

var lgbmTestEval = lgbmModel.Transform(trainTestSplit.TestSet);
var lgbmTestMetric = context.Regression.Evaluate(lgbmTestEval, "y");
Console.WriteLine($"sdca rmse sur testset: {sdcaTestMetric.RootMeanSquaredError}, lgbm rmse sur testset: {lgbmTestMetric.RootMeanSquaredError}");


## Exemple 2 : Régression non linéaire sur LightGbm
Cet exemple montre l'importance de l'optimisation des hyper-paramètres. Tout d'abord, nous créons un jeu de données non linéaire et deux pipelines. Un pipeline a `LightGbm` avec `numberOfLeaves` défini à `10`, l'autre est défini à `1000`. Les deux pipelines sont entraînés avec le même jeu de données d'entraînement et leurs performances sont évaluées sur le même jeu de test.

## Créer un jeu de données non linéaire
Le code ci-dessous crée un jeu de données non linéaire avec un résidu aléatoire. Le jeu de données est chargé dans un ensemble d'entraînement et un ensemble de test.


In [ ]:
var rand = new Random(0);
var context = new MLContext(seed: 1);
var x = Enumerable.Range(-10000, 10000).Select(_x => _x * 0.1f).ToArray();
var y = x.Select(_x => 100 * _x * _x + (rand.NextSingle() - 0.5f) * 10).ToArray();
var df = new DataFrame();
df["X"] = DataFrameColumn.Create("X", x);
df["y"] = DataFrameColumn.Create("y", y);
var trainTestSplit = context.Data.TrainTestSplit(df);
df.Head(10);

var smallLgbmPipeline = context.Transforms.Concatenate("Features", "X")
                            .Append(context.Regression.Trainers.LightGbm("y", numberOfLeaves: 10));

var largeLgbmPipeline = context.Transforms.Concatenate("Features", "X")
                            .Append(context.Regression.Trainers.LightGbm("y", numberOfLeaves: 1000));

var smallLgbmModel = smallLgbmPipeline.Fit(trainTestSplit.TrainSet);
var largeLgbmModel = largeLgbmPipeline.Fit(trainTestSplit.TrainSet);

// évaluation sur le jeu de test
var smallTestEval = smallLgbmModel.Transform(trainTestSplit.TrainSet);
var smallLgbmMetric = context.Regression.Evaluate(smallTestEval, "y");

var largeLgbmEval = largeLgbmModel.Transform(trainTestSplit.TrainSet);
var largeLgbmMetric = context.Regression.Evaluate(largeLgbmEval, "y");

Console.WriteLine($"small lgbm rmse sur testset: {smallLgbmMetric.RootMeanSquaredError}, large lgbm rmse sur testset: {largeLgbmMetric.RootMeanSquaredError}");


## Utiliser AutoML pour simplifier la sélection des entraîneurs et l'optimisation des hyper-paramètres
La sélection des entraîneurs et l'optimisation des hyper-paramètres est un processus important avec beaucoup d'essais et d'erreurs. Ce processus peut être automatisé et simplifié en utilisant les capacités intégrées de `AutoMLExperiment`. `AutoMLExperiment` applique les dernières recherches de Microsoft Research pour effectuer une optimisation des hyper-paramètres rapide, précise et complète avec un budget de temps limité.

Le code ci-dessous montre comment utiliser `AutoMLExperiment` pour trouver le meilleur entraîneur ainsi que ses meilleurs hyper-paramètres sur le jeu de données non linéaire utilisé dans l'Exemple 2. Tout d'abord, un `SweepableEstimatorPipeline` est créé via `context.Auto().Regression("y")`, qui renvoie les régressions les plus populaires avec leur espace de recherche par défaut dans ML.Net. Ensuite, un `AutoMLExperiment` est créé. Il utilise `RootMeanSquaredError` comme métrique d'optimisation et la validation train-test pour évaluer le score des essais, et utilise `NotebookMonitor` pour présenter le processus d'entraînement. Une fois l'entraînement terminé, il renvoie le meilleur essai comme résultat.


In [ ]:
using Microsoft.Data.Analysis;
using System;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Plotly.NET;
using Plotly.NET.CSharp;

// Définir le contexte ML
var context = new MLContext(seed: 1);

// Générer les données
var x = Enumerable.Range(-10000, 10000).Select(_x => _x * 0.1f).ToArray();
var y = x.Select(_x => 100 * _x * _x + (new Random(0).NextDouble() - 0.5) * 10).ToArray();

// Convertir 'y' en Single (float)
var y_single = y.Select(_y => (float)_y).ToArray();

// Créer un DataFrame
var df = new DataFrame();
df["X"] = DataFrameColumn.Create("X", x);
df["Label"] = DataFrameColumn.Create("Label", y_single); // Renommer en 'Label'

// Diviser le DataFrame en ensembles d'entraînement et de test
var trainTestSplit = context.Data.TrainTestSplit(df);

// Définir le pipeline de régression
var pipeline = context.Transforms.Concatenate("Features", "X")
    .Append(context.Auto().Regression("Label"));

// Configurer le moniteur de Notebook
var monitor = new NotebookMonitor(pipeline);
var experiment = context.Auto().CreateExperiment();
experiment.SetPipeline(pipeline)
        .SetRegressionMetric(RegressionMetric.RootMeanSquaredError)
        .SetTrainingTimeInSeconds(30)
        .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
        .SetMonitor(monitor);

// Configurer le visualiseur
monitor.SetUpdate(monitor.Display());

// Exécuter l'expérience
var res = await experiment.RunAsync();

// Obtenir le modèle
var model = res.Model;
var eval = model.Transform(trainTestSplit.TestSet);
var metric = context.Regression.Evaluate(eval, "Label");

// Afficher le résultat
metric.RootMeanSquaredError


# Continuez à apprendre

> [⏩ Module suivant - Évaluation du modèle](./04-Model%20Evaluation.ipynb)  
> [⏪ Module précédent - Préparation des données et ingénierie des fonctionnalités](./02-Data%20Preparation%20and%20Feature%20Engineering.ipynb)
